In [64]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torchvision import transforms

# Here's a simple MLP
class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.flatten(1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Use a standard MNIST normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

dataset = MNIST(root='./data', download=True, transform=transform)
BATCH_SIZE = int(len(dataset)/10) # Big but not too big, because I think that the batch is duplicated for each model in the vmap-enselmble
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, num_workers = 2)

# criterion functional
criterion = F.cross_entropy

#check if we are on GPU
if torch.cuda.is_available():
    NUM_DIRS = 30
    MAX_OOM = 1
    MIN_OOM = -8
else:
    NUM_DIRS = 2 # number of random directions to sample, in addition to the gradient ascent + descent, and radially in + out.
    MAX_OOM = 0 # maximum order of magnitude to sample
    MIN_OOM = -1 # minimum order of magnitude to sample

Define utility functions for parameter space arithmetic. iadd is inplace, all others create a new model.

In [65]:
from copy import deepcopy

@torch.no_grad
def iadd(a:nn.Module, b:nn.Module):
    """add the parameters of b to a, inplace"""
    for a_param, b_param in zip(a.parameters(), b.parameters()):
        a_param.data.add_(b_param.data)
    return a

@torch.no_grad
def add(a_old:nn.Module, b:nn.Module):
    """add the parameters of b to a"""
    a = deepcopy(a_old)
    return iadd(a, b)

@torch.no_grad
def scale(a_old:nn.Module, b:float):
    """scale the parameters of a by b"""
    a = deepcopy(a_old)
    for a_param in a.parameters():
        a_param.data.mul_(b)
    return a

@torch.no_grad
def sub(a:nn.Module, b:nn.Module):
    """subtract the parameters of b from a"""
    neg_b = scale(b, -1)
    return iadd(neg_b, a)

@torch.no_grad
def abs(a:nn.Module):
    """return the norm of the parameters of a"""
    return torch.norm(torch.cat([param.data.flatten() for param in a.parameters()]))

@torch.no_grad
def norm(a:nn.Module):
    """normalize the parameters of a"""
    return scale(a, 1/abs(a))

@torch.no_grad
def rand_like(a:nn.Module):
    """random, normalised direction in the parameter space of a"""
    new = deepcopy(a)
    for param in new.parameters():
        param.data = torch.randn_like(param.data)
    
    return norm(new)

Define profiling context manager: measure execution time and GPU RAM usage before and after.

In [66]:
from time import perf_counter
from contextlib import contextmanager
import torch

@contextmanager
def profiler(description, length=80, pad_char=':'):
    print('\n'+description.center(length, pad_char))
    if torch.cuda.is_available():
        print(f'GPU RAM before execution: Allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB | Reserved: {torch.cuda.memory_reserved()/1e9:.2f} GB')
    start = perf_counter()
    yield
    if torch.cuda.is_available():
        print(f'GPU RAM after execution:  Allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB | Reserved: {torch.cuda.memory_reserved()/1e9:.2f} GB')
    print(f'Finished {description} in {perf_counter() - start:.2f} s'.center(length, pad_char))

with profiler('Profiler example'):
    print('Computation goes here...')


::::::::::::::::::::::::::::::::Profiler example::::::::::::::::::::::::::::::::
Computation goes here...
::::::::::::::::::::::Finished Profiler example in 0.00 s:::::::::::::::::::::::


In [67]:
center_model = SimpleMLP()

# create a model to find the gradient
grad_model = deepcopy(center_model)
grad_model = grad_model.to(device)

optimizer = torch.optim.SGD(grad_model.parameters(), lr=1)
optimizer.zero_grad()
torch.set_grad_enabled(True)

# take a single step of gradient descent
# gradient accumulate over the whole dataset
with profiler('1 step of gradient descent over the whole dataset'):
    for data, target in dataloader:
        data, target = data.to(device), target.to(device)
        output = grad_model(data)
        loss = criterion(output, target)
        loss.backward()

optimizer.step()
# from this point on we wont need any gradients
torch.set_grad_enabled(False)


# calculate the direction of gradient decent from the center model
grad_model = grad_model.to('cpu')
dir_ascent = sub(center_model, grad_model)
dir_ascent = norm(dir_ascent)
dir_descent = scale(dir_ascent, -1)

del grad_model



:::::::::::::::1 step of gradient descent over the whole dataset::::::::::::::::
:::::Finished 1 step of gradient descent over the whole dataset in 14.60 s::::::


In [68]:
radial = norm(center_model)

dirs = {
    'Ascent': dir_ascent,
    'Descent': dir_descent,
    'Radially Out': radial,
    'Radially In': scale(radial,-1),
    }

# add random directions
dirs.update({f'Random {i}': rand_like(center_model) for i in range(NUM_DIRS)})

# check that they are all normalised
assert all(torch.isclose(abs(d), torch.tensor(1.0)) for d in dirs.values())

In [69]:
import pandas as pd
import numpy as np

# convert to base 2
POW_MIN_DIST = int(np.floor(MIN_OOM*np.log2(10)))
POW_MAX_DIST = int(np.ceil(MAX_OOM*np.log2(10)))

# will sample these distances
scales = 2. ** np.arange(POW_MIN_DIST, POW_MAX_DIST + 1)

# offset each direction by a different amount, so that we get a nice spread of distances, to avoid artefacts/robustness
offset_per_direction = 2. ** np.linspace(0, 1, len(dirs), endpoint=False)

# Initialize an empty list to hold tuples for MultiIndex
index_list = []

# Calculate distances using broadcasting
distances = np.outer(scales, offset_per_direction).T.flatten()

# Repeat directions accordingly
directions = np.repeat(list(dirs.keys()), len(scales))

# Create DataFrame directly from numpy arrays
df = pd.DataFrame(index=pd.MultiIndex.from_arrays([directions, distances], names=['Direction', 'Distance']))

df


Empty DataFrame
Columns: []
Index: [(Ascent, 0.0625), (Ascent, 0.125), (Ascent, 0.25), (Ascent, 0.5), (Ascent, 1.0), (Descent, 0.07015387801933581), (Descent, 0.14030775603867163), (Descent, 0.28061551207734325), (Descent, 0.5612310241546865), (Descent, 1.122462048309373), (Radially Out, 0.07874506561842957), (Radially Out, 0.15749013123685915), (Radially Out, 0.3149802624737183), (Radially Out, 0.6299605249474366), (Radially Out, 1.2599210498948732), (Radially In, 0.08838834764831845), (Radially In, 0.1767766952966369), (Radially In, 0.3535533905932738), (Radially In, 0.7071067811865476), (Radially In, 1.4142135623730951), (Random 0, 0.09921256574801246), (Random 0, 0.19842513149602492), (Random 0, 0.39685026299204984), (Random 0, 0.7937005259840997), (Random 0, 1.5874010519681994), (Random 1, 0.11136233976754241), (Random 1, 0.22272467953508482), (Random 1, 0.44544935907016964), (Random 1, 0.8908987181403393), (Random 1, 1.7817974362806785)]

In [70]:
# fill the dataframe with the models
for dir_name, dist in df.index:
    shift = scale(dirs[dir_name], dist)     # shift = direction * distance
    location = iadd(shift, center_model)    # location = center + shift
    df.loc[(dir_name, dist), 'Model'] = location

df

Model
Direction    Distance                                                   
Ascent       0.062500  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             0.125000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             0.250000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             0.500000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             1.000000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
Descent      0.070154  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             0.140308  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             0.280616  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             0.561231  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             1.122462  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
Radially Out 0.078745  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             0.157490  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             0.314980  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             0.629961  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             1.259921  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
Radially In  0.088388  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             0.176777  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             0.353553  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             0.707107  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             1.414214  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
Random 0     0.099213  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             0.198425  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             0.396850  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             0.793701  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             1.587401  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
Random 1     0.111362  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             0.222725  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             0.445449  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             0.890899  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
             1.781797  SimpleMLP(\n  (fc1): Linear(in_features=784, o...

Parallel evaluation with vmap()


In [72]:
from torch.func import stack_module_state, functional_call
from torch import vmap

# add the center model
ensemble_list = [center_model] + list(df['Model'])

# stack to prepare for vmap
params, buffers = stack_module_state(ensemble_list)

with profiler(f'Moving stacked ensemble to {device}'):
    params = {name: tensor.to(device) for name, tensor in params.items()}
    buffers = {name: tensor.to(device) for name, tensor in buffers.items()}

stacked_ensemble = (params, buffers)

# Construct a "stateless" version of one of the models. It is "stateless" in
# the sense that the parameters are meta Tensors and do not have storage.
meta_model = deepcopy(center_model).to('meta')

def meta_model_loss(params_and_buffers, data, target):
    """Compute the loss of a set of params on a batch of data, via the meta model"""
    predictions = functional_call(meta_model, params_and_buffers, (data,))
    predictions = predictions.double() # double precision for better resoltion of small scales
    loss = criterion(predictions, target)
    return loss

# define a loss function that takes the stacked ensemble params, data, and target
# in_dims=(0, None, None) adds an ensemble dimension to the first argument 'params_and_buffers' but not to the other two arguments
vmap_loss = vmap(meta_model_loss, in_dims=(0, None, None))

batch_losses = []
with profiler(f'Evaluating stacked ensemble on {device}'):
    for data, target in dataloader:
        data, target = data.to(device), target.to(device)
        batch_loss = vmap_loss(stacked_ensemble, data, target)
        batch_losses.append(batch_loss)

with profiler('Freeing data and target'):
    del data, target
    torch.cuda.empty_cache()

with profiler('Freeing stacked ensemble'):
    del stacked_ensemble
    torch.cuda.empty_cache()

loss_tensor = torch.mean(torch.stack(batch_losses), dim=0)


:::::::::::::::::::::::::Moving stacked ensemble to cpu:::::::::::::::::::::::::
:::::::::::::::Finished Moving stacked ensemble to cpu in 0.00 s::::::::::::::::

:::::::::::::::::::::::Evaluating stacked ensemble on cpu:::::::::::::::::::::::
:::::::::::::Finished Evaluating stacked ensemble on cpu in 25.68 s:::::::::::::


In [73]:
# Convert the loss tensor to a list and unpack
center_loss, *dir_losses = loss_tensor.tolist()

# Ensure the length of dir_losses matches the DataFrame length
assert len(dir_losses) == len(df)

# Add the directional losses to the DataFrame
df['Loss'] = dir_losses

# Add the center_loss to each Direction, for Distance=0
for direction in df.index.get_level_values('Direction').unique():
    df.loc[(direction, 0.0), 'Loss'] = center_loss

df.sort_index(inplace=True)

df

Model  \
Direction    Distance                                                      
Ascent       0.000000                                                NaN   
             0.062500  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.125000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.250000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.500000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             1.000000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
Descent      0.000000                                                NaN   
             0.070154  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.140308  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.280616  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.561231  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             1.122462  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
Radially In  0.000000                                                NaN   
             0.088388  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.176777  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.353553  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.707107  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             1.414214  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
Radially Out 0.000000                                                NaN   
             0.078745  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.157490  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.314980  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.629961  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             1.259921  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
Random 0     0.000000                                                NaN   
             0.099213  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.198425  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.396850  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.793701  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             1.587401  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
Random 1     0.000000                                                NaN   
             0.111362  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.222725  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.445449  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.890899  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             1.781797  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   

                           Loss  
Direction    Distance            
Ascent       0.000000  2.312136  
             0.062500  2.340928  
             0.125000  2.370135  
             0.250000  2.428125  
             0.500000  2.541591  
             1.000000  2.788666  
Descent      0.000000  2.312136  
             0.070154  2.281144  
             0.140308  2.252334  
             0.280616  2.202327  
             0.561231  2.124990  
             1.122462  1.998361  
Radially In  0.000000  2.312136  
             0.088388  2.311847  
             0.176777  2.311567  
             0.353553  2.311031  
             0.707107  2.310054  
             1.414214  2.308425  
Radially Out 0.000000  2.312136  
             0.078745  2.312401  
             0.157490  2.312674  
             0.314980  2.313243  
             0.629961  2.314478  
             1.259921  2.317397  
Random 0     0.000000  2.312136  
             0.099213  2.312078  
             0.198425  2.312020  
             0.396850  2.311905  
             0.793701  2.311691  
             1.587401  2.311424  
Random 1     0.000000  2.312136  
             0.111362  2.312071  
             0.222725 

In [74]:
import plotly.graph_objects as go

# Create a Plotly figure
fig = go.Figure()

# Plotting directly from grouped data
for direction, data in df.groupby(level='Direction'):
    fig.add_trace(go.Scatter(
        x=data.index.get_level_values('Distance'),
        y=data['Loss'],
        mode='lines+markers',
        name=direction
    ))

# Set axis labels and title with adjusted figure dimensions
fig.update_layout(
    xaxis_title='Distance from Center in Parameter Space',
    yaxis_title='Loss',
    title=f'Loss Landscape of {center_model.__class__.__name__} on {dataset.__class__.__name__}',
    legend_title='Direction',
    template='seaborn',
    height=600,
)


# Show the figure
fig.show()


Now let's measure the roughness.

In [75]:
# TODO: Add nice explanation of the roughness calculation
df['Roughness'] = np.nan

# Calculate the roughness, one direction at a time
for direction, group in df.groupby(level='Direction'):
    distA = group.index.get_level_values('Distance').to_numpy()
    losses = group['Loss'].to_numpy()

    # Named slices, so that 
    # distA[x] really corresponds to the distance between A and X ∈ {A, B, C},
    # losses[x] really corresponds to the loss at X ∈ {A, B, C}
    a = 0
    b = slice(1, -1)
    c = slice(2, None)

    # Ensure distances and slices are correct
    assert np.all(distA[c] == 2 * distA[b]), "Distance check failed"

    # Calculate paths between A, B, and C
    AB = np.sqrt((losses[b] - losses[a]) ** 2 + distA[b] ** 2)
    BC = np.sqrt((losses[c] - losses[b]) ** 2 + distA[b] ** 2)
    AC = np.sqrt((losses[c] - losses[a]) ** 2 + distA[c] ** 2)

    # Roghness is how much the path via B (A->B->C) is longer than the direct path A->C
    roughness = (AB + BC) / AC

    # Update DataFrame with roughness values
    df.loc[(direction,), 'Roughness'] = np.concatenate([[np.nan], roughness, [np.nan]])

df

Model  \
Direction    Distance                                                      
Ascent       0.000000                                                NaN   
             0.062500  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.125000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.250000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.500000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             1.000000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
Descent      0.000000                                                NaN   
             0.070154  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.140308  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.280616  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.561231  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             1.122462  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
Radially In  0.000000                                                NaN   
             0.088388  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.176777  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.353553  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.707107  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             1.414214  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
Radially Out 0.000000                                                NaN   
             0.078745  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.157490  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.314980  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.629961  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             1.259921  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
Random 0     0.000000                                                NaN   
             0.099213  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.198425  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.396850  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.793701  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             1.587401  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
Random 1     0.000000                                                NaN   
             0.111362  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.222725  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.445449  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             0.890899  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
             1.781797  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   

                           Loss  Roughness  
Direction    Distance                       
Ascent       0.000000  2.312136        NaN  
             0.062500  2.340928   1.000004  
             0.125000  2.370135   1.000000  
             0.250000  2.428125   1.000009  
             0.500000  2.541591   1.000103  
             1.000000  2.788666        NaN  
Descent      0.000000  2.312136        NaN  
             0.070154  2.281144   1.000087  
             0.140308  2.252334   1.000458  
             0.280616  2.202327   1.001355  
             0.561231  2.124990   1.001250  
             1.122462  1.998361        NaN  
Radially In  0.000000  2.312136        NaN  
             0.088388  2.311847   1.000000  
             0.176777  2.311567   1.000000  
             0.353553  2.311031   1.000000  
             0.707107  2.310054   1.000000  
             1.414214  2.308425        NaN  
Radially Out 0.000000  2.312136        NaN  
             0.078745  2.312401   1.000000  
             0.157490  2.312674   1.000000  
             0.314980  2.313243   1.000000  
             0.629961  2.314478   1.000000  
             1.259921  2.317397        NaN  
Random 0

In [76]:
# Create a Plotly figure
fig = go.Figure()

# Plotting roughness for each direction using grouped data
for direction, data in df.groupby(level='Direction'):
    fig.add_trace(go.Scatter(
        x=data.index.get_level_values('Distance'),
        y=data['Roughness'],
        mode='markers',
        name=direction
    ))

# Set axis labels, title, and configure log scale on x-axis
fig.update_layout(
    xaxis_title='Coarse Graining Scale',
    yaxis_title='Roughness',
    title=f'Scale Dependent Roughness of {center_model.__class__.__name__} on {dataset.__class__.__name__}',
    legend_title='Direction',
    template='seaborn',
    xaxis={
        'type': 'log',
        'dtick': 1,  # Tick every power of ten
    },
    height=600
)

# Display the figure
fig.show()
